### 인공신경망 (ANN) 모델 살펴보기
- 구성 : 입력층 + 은닉층 + 출력층
    * 입력층 : 입력값 <-- 입력 피처수, 출력값 <--- 층의 퍼셉트론 수 
    * 출력층 : 입력값 <--- 이전층의 결과수, 출력값 <--- 입력 타겟 수 (이진분류 => 1개, 다중분류 => 클래스수, 회귀 => 1개)

In [2106]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2107]:
torch.random.manual_seed(12)

In [2108]:
# 데이터 생성
# - 피처 : 점수, 공부시간
# - 타겟 : 합격, 불합격   ===>  2진 분류
featureTS = torch.FloatTensor([[40, 4], [60, 5], [80, 6], [100, 8]])
targetTS = torch.FloatTensor([[0], [1], [1], [1]])

print(f"[featureTS] {featureTS.shape}, {featureTS.ndim}D")
print(f"[targetTS] {targetTS.shape}, {targetTS.ndim}D")

[featureTS] torch.Size([4, 2]), 2D
[targetTS] torch.Size([4, 1]), 2D


In [2109]:
# ANN 모델
class Net(nn.Module):
    
    # 모델 구조 설정
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 4)    # 입력층
        self.sig1 = nn.Sigmoid()
        
        self.layer2 = nn.Linear(4, 4)    # 은닉층
        self.sig2 = nn.Sigmoid()
        
        self.layer3 = nn.Linear(4, 1)    # 출력층
        self.sig3 = nn.Sigmoid()
        # sigmoid는 1개만 만들어도 됨! 먼가 강의를 위해 여러개 만드신둡
        
    # 순전파 함수
    def forward(self, x):
        y = self.layer1(x)
        y = self.sig1(y)
        
        y = self.layer2(y)
        y = self.sig2(y)
        
        y = self.layer3(y)
        y_total = self.sig3(y)
        
        return y_total

In [2110]:
# 모델 인스턴스 설정
model1 = Net()
model1

Net(
  (layer1): Linear(in_features=2, out_features=4, bias=True)
  (sig1): Sigmoid()
  (layer2): Linear(in_features=4, out_features=4, bias=True)
  (sig2): Sigmoid()
  (layer3): Linear(in_features=4, out_features=1, bias=True)
  (sig3): Sigmoid()
)

In [2111]:
for param in model1.named_parameters():
    print(param)
    print('============')
    
# model1.named_parameters() ===> 이게 generator라서 for문 돌면서 1개씩 빼는 것이 가능하다.
# 퍼셉트론 : 다수의 신호를 받아서 한개의 신호를 출력한다.
# - 고로 1개의 퍼셉트론에 1세트의 W와 1개의 b가 나온다.
#    (여기서 세트라 함은 피처의 수를 말한다. 여기서는 피처의 수가 2개이므로 1세트가 2개이다.)


('layer1.weight', Parameter containing:
tensor([[-0.0485, -0.3779],
        [-0.0669,  0.1232],
        [-0.1292, -0.5273],
        [ 0.1941, -0.3648]], requires_grad=True))
('layer1.bias', Parameter containing:
tensor([ 0.3270,  0.3146, -0.4253,  0.2755], requires_grad=True))
('layer2.weight', Parameter containing:
tensor([[ 0.0830,  0.1318,  0.0559, -0.3738],
        [ 0.4790,  0.3443, -0.3744, -0.0544],
        [ 0.1601, -0.4446, -0.3427,  0.3137],
        [ 0.2216, -0.2283, -0.1997,  0.1099]], requires_grad=True))
('layer2.bias', Parameter containing:
tensor([ 0.0784,  0.1083, -0.0661,  0.3813], requires_grad=True))
('layer3.weight', Parameter containing:
tensor([[-0.1784, -0.2396, -0.2434, -0.3128]], requires_grad=True))
('layer3.bias', Parameter containing:
tensor([0.1423], requires_grad=True))


In [2112]:
# 학습 진행
# - 에포크 : 데이터 처음부터 끝까지 한번 학습
# - 배치사이즈 : 데이터를 일정한 크기로 자른 것 (일반적으로 32개 많이 함)

# 1 에포크를 들려보았다. (처음부터 끝까지 1번 학습 진행해본것!. 배치사이즈는 음슴)
output = model1(featureTS)
output

tensor([[0.4043],
        [0.4043],
        [0.4043],
        [0.4043]], grad_fn=<SigmoidBackward0>)

In [2113]:
# 손실 계산 => 2진 분류 --> 0.5 기준으로 판별
correct = sum((output > 0.5) == targetTS )           # 모델을 통해서 예측한 값에서 실제 타겟과 같은지 계산해보지 ==> 이것이 손실 계산
correct

tensor([1])

In [2114]:
# 정확도 (Accuracy)
accuracy = correct / targetTS.shape[0]

print(f"Accuracy: {accuracy.item()}")

# 4개중에 3개 맞췄으니까 0.75 나옴

Accuracy: 0.25


In [2115]:
# 정밀도 (prcision)
correct = sum(output>0.5)
if not correct:
    print(f"정밀도 : 0.0")

정밀도 : 0.0


In [2116]:
# 재현율
correct = sum((output>0.5) == targetTS)
correct

tensor([1])

### 손실함수 즉, 정답과 예측값 사이의 오차
- 회귀 : MSE, MAE, RMSE, ...
- 분류 : binary_cross_entropy, cross_entropy

In [2117]:
print(f"targetTS \n {targetTS}")
print(f"output \n {output}")

targetTS 
 tensor([[0.],
        [1.],
        [1.],
        [1.]])
output 
 tensor([[0.4043],
        [0.4043],
        [0.4043],
        [0.4043]], grad_fn=<SigmoidBackward0>)


In [2118]:
F.binary_cross_entropy(output, targetTS)

# binary는 활성함수 거친 것을 넣어주어야함

tensor(0.8086, grad_fn=<BinaryCrossEntropyBackward0>)

In [2119]:
# F.cross_entropy()
# cross는 활성함수 거치지 않은 것을 넣어주어야 함. (애는 logit 타입을 원함)

### 학습 진행 <hr>
- (1) 모델로 학습 진행  ==> 학습 횟수 지정
- (2) 학습 결과와 정답 비교 손실 계산  ==> 손실 함수가 해줌 
- (3) W, b 업데이트 ==> 옵티마이저가 해줌 

In [2120]:
### ===> 학습에 필요한 준비
model2 = Net()
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model2.parameters())
EPOCHS = 10

In [2121]:
### ===> 학습 진행
# 학습 모드로 모델 설정
model2.train()         # 배치 정규화, Dropout, 가중치 초기화 등의 작업을 할 수 있는 환경/메모리 만들어줌 (준비 상태 만들어줌)

for ep in range(EPOCHS):

    # 학습
    output = model2(featureTS)
    
    # 손실 계산
    loss = loss_fn(output, targetTS)
    
    # 업데이트    
    optimizer.zero_grad()    # 텐서의 grad 속성에 값을 초기화 
                             #  (그 전 epoch 값에서 계산된 grad 값이 초기화 된다는 거긔)
    loss.backward()          # 러닝레이트와 손실값으로 새로운 W, b 계산 진행
    optimizer.step()         # 모델로부터 전달받은 W, b 텐서의 주소로 새로운 W, b로 업데이트 
    
    # 결과물 확인해보자
    print(f"ep => {ep}    loss => {loss.item()}")

ep => 0    loss => 0.8989731073379517
ep => 1    loss => 0.897470235824585
ep => 2    loss => 0.8959712982177734
ep => 3    loss => 0.8944765329360962
ep => 4    loss => 0.8929858207702637
ep => 5    loss => 0.8914993405342102
ep => 6    loss => 0.8900171518325806
ep => 7    loss => 0.88853919506073
ep => 8    loss => 0.8870657682418823
ep => 9    loss => 0.8855966925621033


In [2122]:
### ===> 테스트 : 모델의 성능 평가 / 업데이트 발생하면 안됨 !
### ===> 모델 동작 모드 설정, autograd 기능 정지, requires_grad 텐서 정지 

# 테스트 모드로 모델 설정
model2.eval()            # train에서 갖추어진 환경이 필요없으니까 그 모드를 끈거임!

with torch.no_grad():     # 미연의 방지를 위함 ^^ ㅎㅎ
    # 학습
    output = model2(featureTS)

    # 손실 계산 : 테스트 데이터에 정답 (타겟) 존재하는 경우 진행
    loss = loss_fn(output, targetTS)

    # 결과물 확인해보자
    print(f"[TEST] loss => {loss.item()}")

# loss => 0.8841320276260376

In [2123]:
# 테스트 모드로 모델 설정
model2.eval()

# 학습
output = model2(featureTS)

# 손실 계산 : 테스트 데이터에 정답 (타겟) 존재하는 경우 진행
loss = loss_fn(output, targetTS)

# 결과물 확인해보자
print(f"[TEST] loss => {loss.item()}")

[TEST] loss => 0.8841320276260376
